In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
data = pd.read_csv("/content/drive/MyDrive/Capstone Project/worksheet_graphs - Sheet1 (1).csv")

In [3]:
#print(data.head())

In [4]:
cleaned_data = data.dropna()

cleaned_data.reset_index(drop=True, inplace=True)

data = cleaned_data


# Chi-Square Test for Independence
Used to determine if there is a significant association between two categorical variables for eg. categorical variables like 'Car Mode' and 'Vehicle Damage'.

In [5]:
import scipy.stats as stats

In [6]:
# Specify the columns you're interested in testing against 'Vehicle Damage'
columns_to_test = [
    'Vehicle 1 was Moving', 'Number of Vehicles involved in Accident (w V1)',
    'Vehicle 2 was Moving', 'Number of Vehicles involved in Accident (w V2)',
    'Car Mode', 'Weather Vehicle 1', 'Weather Vehicle 2',
    'Lighting Vehicle 1', 'Lighting Vehicle 2', 'Roadway Surface Vehicle 1',
    'Roadway Surface Vehicle 2', 'Roadway Conditions Vehicle 1',
    'Roadway Conditions Vehicle 2', 'Movement Preceding Collision Vehicle 1',
    'Movement Preceding Collision Vehicle 2', 'Type of Collison Vehicle 1',
    'Type of Collison Vehicle 2'
]

In [7]:
# Iterate over each column and perform Chi-Square Test
results = []
for column in columns_to_test:
    contingency_table = pd.crosstab(cleaned_data[column], cleaned_data['Vehicle Damage'])
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
    results.append((column, chi2, p_value, dof))



In [8]:
# Convert results to a DataFrame for better visualization
result_df = pd.DataFrame(results, columns=['Column', 'Chi-Square Statistic', 'P-value', 'Degrees of Freedom'])

# Display the results
print(result_df)

                                            Column  Chi-Square Statistic  \
0                             Vehicle 1 was Moving              0.908145   
1   Number of Vehicles involved in Accident (w V1)              3.974395   
2                             Vehicle 2 was Moving              2.025679   
3   Number of Vehicles involved in Accident (w V2)              2.598746   
4                                         Car Mode              0.902937   
5                                Weather Vehicle 1             10.859083   
6                                Weather Vehicle 2             10.143427   
7                               Lighting Vehicle 1              6.354899   
8                               Lighting Vehicle 2              5.752518   
9                        Roadway Surface Vehicle 1              4.909849   
10                       Roadway Surface Vehicle 2              2.297175   
11                    Roadway Conditions Vehicle 1              8.900118   
12          

In [30]:
df = pd.DataFrame(result_df)

In [31]:
# Save DataFrame to CSV
df.to_csv('bayesian.csv', index=False)

# Download the file
files.download('bayesian.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#StatsModels

Needs numeric data

In [9]:
#pip install statsmodels


In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [11]:
data['Car Conditions'] = data[['Vehicle 1 was Moving', 'Number of Vehicles involved in Accident (w V1)', 'Movement Preceding Collision Vehicle 1', 'Car Mode']].apply(lambda x: '_'.join(x.astype(str)), axis=1)
data['External Conditions'] = data[['Weather Vehicle 1', 'Lighting Vehicle 1', 'Roadway Surface Vehicle 1', 'Roadway Conditions Vehicle 1']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [12]:
data['Vehicle Damage'] = data['Vehicle Damage'].astype('category')


In [13]:
formula = 'Q("Vehicle Damage") ~ Q("Car Conditions") + Q("External Conditions")'

# Assuming Vehicle Damage is categorical and using Car Mode for random effects for simplicity
data['Vehicle Damage'] = data['Vehicle Damage'].astype('category')

# Fit the model
model = smf.mixedlm(formula, data, groups=data['Car Mode'], family=sm.families.Binomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

ValueError: endog has evaluated to an array with multiple columns that has shape (324, 4). This occurs when the variable converted to endog is non-numeric (e.g., bool or str).

#Bayesian Modeling with PyMC3

needs to be used on conda or virtual env

In [15]:
# !pip install theano
# !pip install arviz
# !pip install pymc3

In [16]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'


In [24]:
!pip install --upgrade numpy==1.20.3

  Using cached numpy-1.20.3.zip (7.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [21]:
import theano
from theano import tensor as tt

# Simple Theano test
x = tt.scalar('x')
y = tt.scalar('y')
z = x + y
f = theano.function([x, y], z)
print("Testing Theano computation:", f(2, 3))  # Should output 5

AttributeError: module 'numpy.distutils.__config__' has no attribute 'blas_opt_info'

In [17]:
import theano
theano.config.blas.check_openmp = False

AttributeError: module 'numpy.distutils.__config__' has no attribute 'blas_opt_info'

In [ ]:
import pymc3 as pm
import theano.tensor as tt
print("PyMC3 version:", pm.__version__)
print("Theano version:", tt.config.version)

In [ ]:
data['Vehicle Damage'] = pd.Categorical(data['Vehicle Damage'], categories=["None", "Minor", "Moderate", "Major"], ordered=True)
data['Car Conditions'] = pd.Categorical(data['Car Conditions'])
data['External Conditions'] = pd.Categorical(data['External Conditions'])

# Convert categories to codes
damage_codes = data['Vehicle Damage'].cat.codes
car_conditions_codes = pd.get_dummies(data['Car Conditions'], drop_first=True)
external_conditions_codes = pd.get_dummies(data['External Conditions'], drop_first=True)

In [ ]:
with pm.Model() as model:
    # Priors for ordinal thresholds (cutpoints)
    cutpoints = pm.Normal("cutpoints", mu=0, sigma=1.5, shape=3, transform=pm.distributions.transforms.ordered)

    # Priors for coefficients
    beta_car = pm.Normal("beta_car", mu=0, sigma=1, shape=car_conditions_codes.shape[1])
    beta_ext = pm.Normal("beta_ext", mu=0, sigma=1, shape=external_conditions_codes.shape[1])

    # Linear model
    mu = (tt.dot(car_conditions_codes.values, beta_car) +
          tt.dot(external_conditions_codes.values, beta_ext))

    # Model likelihood
    Y_obs = pm.OrderedLogistic("Y_obs", mu=mu, cutpoints=cutpoints, observed=damage_codes)

    # Sample from the posterior
    trace = pm.sample(1000, return_inferencedata=True)


In [25]:
df = pd.DataFrame(data)

In [27]:
from google.colab import files

In [29]:
# Save DataFrame to CSV
df.to_csv('bayesian.csv', index=False)

# Download the file
files.download('bayesian.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#New

In [3]:
import statsmodels.api as sm
import numpy as np

In [17]:
print(data.columns)


Index(['File ID', 'Vehicle 1 Manufacturer', 'Vehicle 1 Business Name', 'Date',
       'Time', 'Vehicle 1 Year of Manufacturing', 'Vehicle 1 Make',
       'Vehicle 1 Model', 'Location of Accident', 'City of Accident',
       'County of Accident', 'State of Accident', 'ZIPCODE of Accident',
       'Vehicle 1 was Moving', 'Vehicle 1 was Stopped in Traffic',
       'Pedestrian Involved in Accident (w V1)',
       'Bicyclist involved in Accident (w V1)',
       'Any other involved entity (w V1)',
       'Number of Vehicles involved in Accident (w V1)', 'Vehicle Damage',
       'RB', 'LR1', 'LR2', 'LR3', 'RR1', 'RR2', 'RR3', 'LRP1', 'LRP2', 'LRP3',
       'LRP4', 'RRP1', 'RRP2', 'RRP3', 'RRP4', 'FDS1', 'FDS2', 'FDS3', 'FDS4',
       'FPS1', 'FPS2', 'FPS3', 'FPS4', 'LFC1', 'LFC2', 'LFC3', 'RFC1', 'RFC2',
       'RFC3', 'FB', 'Vehicle 2 Model', 'Vehicle 2 Year of Manufacturing',
       'Vehicle 2 was Moving', 'Vehicle 2 was Stopped in Traffic',
       'Pedestrian Involved in Accident (w V2)',


In [10]:
data = pd.get_dummies(data, drop_first=True)

# Ensuring all explanatory variables are numeric
X = data.drop('Vehicle Damage', axis=1).apply(pd.to_numeric, errors='coerce')  # Replace 'Vehicle_Damage' with the correct name
y = data['Vehicle Damage'].apply(pd.to_numeric, errors='coerce')  # Modify this line based on your actual target variable


# Drop any columns that still have non-numeric types
X = X.select_dtypes(include=[np.number])
y = y.dropna()  # Ensuring the target variable has no NaNs

# Checking if X and y have the same length
X = X.loc[y.index]

# Fit a GLM
model = sm.GLM(y, sm.add_constant(X), family=sm.families.Binomial())
result = model.fit()

print(result.summary())

KeyError: "['Vehicle Damage'] not found in axis"

In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
data['Car Conditions'] = data[['Vehicle 1 was Moving', 'Number of Vehicles involved in Accident (w V1)', 'Movement Preceding Collision Vehicle 1', 'Car Mode']].apply(lambda x: '_'.join(x.astype(str)), axis=1)
data['External Conditions'] = data[['Weather Vehicle 1', 'Lighting Vehicle 1', 'Roadway Surface Vehicle 1', 'Roadway Conditions Vehicle 1']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [19]:
model = smf.mixedlm("Vehicle Damage ~ Weather + Lighting + Road_Conditions", data,
                    groups=data["Vehicle 1 Manufacturer"])
result = model.fit()
print(result.summary())

SyntaxError: invalid syntax (<unknown>, line 1)